In [26]:
import os
import numpy as np
import tensorflow as tf
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

# GPU check
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  0


In [28]:
data_dir = r"C:\Users\klair\Machine Learning\CV Projects\Animal Classifiction\Animal Classification\dataset"  # Replace with your dataset path

# Get all file paths and labels
filepaths, labels = [], []
class_names = sorted(os.listdir(data_dir))
for idx, class_name in enumerate(class_names):
    class_dir = os.path.join(data_dir, class_name)
    for file in os.listdir(class_dir):
        filepaths.append(os.path.join(class_dir, file))
        labels.append(idx)

filepaths = np.array(filepaths)
labels = np.array(labels)
num_classes = len(class_names)
print("Classes:", class_names)
print("Total images:", len(filepaths))


Classes: ['Bear', 'Bird', 'Cat', 'Cow', 'Deer', 'Dog', 'Dolphin', 'Elephant', 'Giraffe', 'Horse', 'Kangaroo', 'Lion', 'Panda', 'Tiger', 'Zebra']
Total images: 1944


In [29]:
n_splits = 3
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
fold_accuracies = []


In [ ]:
train_gen = train_datagen.flow_from_dataframe(
    dataframe=pd.DataFrame({
        'filename': train_files,
        'class': train_labels.astype(str)   # <- convert to string
    }),
    x_col='filename',
    y_col='class',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='sparse',
    shuffle=True
)

val_gen = val_datagen.flow_from_dataframe(
    dataframe=pd.DataFrame({
        'filename': val_files,
        'class': val_labels.astype(str)   # <- convert to string
    }),
    x_col='filename',
    y_col='class',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='sparse',
    shuffle=False
)


===== Fold 1/3 =====


TypeError: If class_mode="sparse", y_col="class" column values must be strings.

In [24]:
# Get list of all image filepaths and labels
filepaths = []
labels = []
class_names = sorted(os.listdir(data_dir))

for idx, class_name in enumerate(class_names):
    class_dir = os.path.join(data_dir, class_name)
    for file in os.listdir(class_dir):
        filepaths.append(os.path.join(class_dir, file))
        labels.append(idx)

filepaths = np.array(filepaths)
labels = np.array(labels)

kf = KFold(n_splits=3, shuffle=True, random_state=42)
fold_accuracies = []


In [31]:
for fold, (train_idx, val_idx) in enumerate(kf.split(filepaths, labels), 1):
    print(f"===== Fold {fold} / 3 =====")
    
    train_files = filepaths[train_idx]
    train_labels = labels[train_idx]
    val_files = filepaths[val_idx]
    val_labels = labels[val_idx]
    
    train_gen = datagen.flow_from_dataframe(
        dataframe=pd.DataFrame({'filename': train_files, 'class': train_labels}),
        x_col='filename',
        y_col='class',
        target_size=(img_size,img_size),
        batch_size=batch_size,
        class_mode='raw',
        shuffle=True
    )
    
    val_gen = datagen.flow_from_dataframe(
        dataframe=pd.DataFrame({'filename': val_files, 'class': val_labels}),
        x_col='filename',
        y_col='class',
        target_size=(img_size,img_size),
        batch_size=batch_size,
        class_mode='raw',
        shuffle=False
    )
    
    # Callbacks
    early_stop = EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2)
    
    # Create model
    model, base_model = create_model(img_size, num_classes)
    
    # Train with frozen base
    history = model.fit(
        train_gen,
        validation_data=val_gen,
        epochs=epochs_frozen,
        callbacks=[early_stop, reduce_lr],
        verbose=1
    )
    
    # Fine-tune top layers
    base_model.trainable = True
    for layer in base_model.layers[:-50]:  # Freeze first layers
        layer.trainable = False
    
    model.compile(optimizer=Adam(1e-4),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    history_ft = model.fit(
        train_gen,
        validation_data=val_gen,
        epochs=epochs_finetune,
        callbacks=[early_stop, reduce_lr],
        verbose=1
    )
    
    acc = model.evaluate(val_gen, verbose=0)[1]
    print(f"Fold {fold} Accuracy: {acc:.4f}")
    fold_accuracies.append(acc)

# Cross-validation results
print("\n===== Cross Validation Results =====")
for i, acc in enumerate(fold_accuracies):
    print(f"Fold {i+1}: {acc:.4f}")
print(f"Mean Accuracy: {np.mean(fold_accuracies):.4f}")


===== Fold 1 / 3 =====
Found 1296 validated image filenames.
Found 648 validated image filenames.


c:\Users\klair\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 27s 533ms/step - accuracy: 0.1609 - loss: 3.0717 - val_accuracy: 0.6327 - val_loss: 1.3242 - learning_rate: 0.0010
Epoch 2/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 19s 478ms/step - accuracy: 0.5652 - loss: 1.4862 - val_accuracy: 0.7454 - val_loss: 0.8980 - learning_rate: 0.0010
Epoch 3/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 19s 475ms/step - accuracy: 0.6712 - loss: 1.0704 - val_accuracy: 0.7901 - val_loss: 0.7455 - learning_rate: 0.0010
Epoch 4/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 19s 477ms/step - accuracy: 0.7432 - loss: 0.8839 - val_accuracy: 0.8148 - val_loss: 0.6536 - learning_rate: 0.0010
Epoch 5/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 19s 478ms/step - accuracy: 0.7623 - loss: 0.7729 - val_accuracy: 0.8025 - val_loss: 0.6419 - learning_rate: 0.0010
Epoch 6/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 19s 476ms/step - accuracy: 0.7945 - loss: 0.6788 - val_accuracy: 0.8380 - val_loss: 0.5811 - learning_rate: 0.0010
Epoch 7/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 19s 476ms/step - accuracy: 0.8027 - loss: 0.